In [1]:
from pymisca.util import *
from pymisca.vis_util import *
import IPython.display as ipd

import KBs
import random

DIR = 'data_rand'
!mkdir -p {DIR}


is in ipython: 1
[WARN] pymisca.vis_util cannot find network


In [2]:
from KBs import *
ps = np.array([0.1   , 0.01  , 0.001 , 0.0001, 0.    ]) 
def main(env,ps=ps,agg=np.min):
#     ps = np.array([0.1   , 0.01  , 0.001 , 0.0001, 0.    ])
    lst = []
    for pr in ps:
        adv = mix_adv(fill_ber,env.adv,pa=pr)
        hist = sample(env,adv=adv)
        hist = hist.reshape(hist.shape[:-2]+(-1,))
        dlst=  []
        for x in hist:
            d= spdist.pdist(x,'hamming')
            d = hill(d)
            D = spdist.squareform(d)
            dlst+=[D]        
            np.fill_diagonal(D,2)    
        lst.append(agg(dlst,axis=(-1)))
    lst=  np.array(lst)
    
    opt = post(lst)
    out = np.vstack([[ps],opt])
    return out
def post(o,ps=None,ylim=[0,150],axs = None,silent=0,lab=''):
    arr = np.mean(o,axis=-1)
    out = map(lambda x:fit_exp(x)[0],arr) 
    #### out = (popt,yfit,loss)
    return zip(*out)
# def worker0325(**kwargs):
def worker0325(dct):
    env = KBs.guess(dct=dct)
    env.change_size((128,128,32**2))
    out = main(env)
    return out

20 rule per min

bsize/20 = 2 min

In [ ]:
%%time
import os
import workers
workers.worker0325 = worker0325
import time
import warnings
# warnings.filterwarnings("once")
warnings.filterwarnings("ignore")

ts = []
t0 = time.time()
oit = list(range(0,60*24))
for seed in oit:
    bsize= 40
    worker = workers.worker0325
    kb = KBs.kb_2dntca()
    it = kb.bulk_rstr(seed=seed,bsize=bsize)
    out = mp_map(worker,it,n_cpu=6)

    fname = os.path.join(DIR,'seed%d_size%d.npy'%(seed,bsize))
    np.save(fname,out)
    
    print "finished %d of %d"%(seed,len(oit))
    t = time.time()
    ts.append(t-t0)
    t0 = t
#     break
print 'finished'
print 'avgtime:%.5fs'%(np.mean(ts))

finished 0 of 1440
finished 1 of 1440
